In [2]:
import sys
sys.path.append('../..')

import torch
from torch import nn
from torch.utils.data import DataLoader
#from src.preprocessing.dictionary import Dictionary
#from data.dataset import Dataset
from src.models import RNN_Decoder, RNN_Encoder, AttentionLayer

# Define hyperparameters
source_vocab_size = 1000
target_vocab_size = 2000
hidden_size = 256
encoder_layers = 2
decoder_layers = 2
dropout = 0.2
embedding_dim = 128
max_length_src = 10
max_length_trg = 9
batch_size = 128

PADDING=-1



/Users/lukas/anaconda3/envs/MT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

# Create dummy input data
encoder_input = torch.randint(0, source_vocab_size, (batch_size, max_length_src))
decoder_input = torch.randint(0, target_vocab_size, (batch_size,))
target_tensor = torch.randint(0, target_vocab_size, (batch_size, max_length_trg))
encoder_lens = torch.randint(1,max_length_src,(batch_size,))
encoder_lens[1] = max_length_src
print(encoder_input.shape)
print(decoder_input.shape)
print(target_tensor.shape)
# create lens for the input data one entry for every element in batch with max length
#create lens tensor for encoder with first dim batch size and one entry for every element in batch with max length

src_pad_mask = (encoder_input == PADDING)
print(encoder_lens.shape)
# Create encoder, decoder, and attention instances
encoder = RNN_Encoder(source_vocab_size=source_vocab_size, lstm_hidden_size=hidden_size, num_lstm_layers=encoder_layers, lstm_dropout_rate=dropout, bidirectional=True, embedding_dim=embedding_dim)
decoder = RNN_Decoder(target_vocab_size, hidden_size, hidden_size*2, decoder_layers, dropout, embedding_dim, attention_function='dot_product', teacher_forcing_ratio=0.5)

torch.Size([128, 10])
torch.Size([128])
torch.Size([128, 9])
torch.Size([128])


In [5]:
# Test encoder
(encoder_output,lens), (h, c) = encoder(encoder_input,encoder_lens)
print("Encoder output shape:", encoder_output.shape)
print("Encoder hidden state shape:", h.shape)
print("Encoder cell state shape:", c.shape)
print("Lens:", lens.shape)

Encoder output shape: torch.Size([128, 10, 512])
Encoder hidden state shape: torch.Size([4, 128, 256])
Encoder cell state shape: torch.Size([4, 128, 256])
Lens: torch.Size([128])


In [6]:
# create batchdim x 1 tensor as input for softmax dummy tensor
probabilities = torch.randint(0, 6, (5,10)).float()
print(decoder_input.shape)
# Test decoder
probabilities = torch.nn.functional.softmax(probabilities)
_, index = torch.max(probabilities, dim=1)
print(index.shape)
print(probabilities)
print(index.unsqueeze(1).shape)

torch.Size([128])
torch.Size([5])
tensor([[0.2973, 0.0148, 0.1094, 0.1094, 0.2973, 0.0054, 0.1094, 0.0148, 0.0020,
         0.0402],
        [0.0105, 0.0039, 0.0105, 0.0039, 0.0775, 0.0039, 0.2108, 0.5730, 0.0285,
         0.0775],
        [0.3934, 0.1447, 0.0072, 0.0196, 0.0532, 0.1447, 0.1447, 0.0532, 0.0196,
         0.0196],
        [0.0124, 0.2495, 0.2495, 0.0338, 0.0918, 0.2495, 0.0918, 0.0124, 0.0046,
         0.0046],
        [0.2680, 0.2680, 0.0049, 0.0986, 0.0018, 0.0363, 0.0363, 0.0133, 0.0049,
         0.2680]])
torch.Size([5, 1])


/var/folders/4l/tkchfljx33xg05m__rqx8xpw0000gn/T/ipykernel_20875/1131713800.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = torch.nn.functional.softmax(probabilities)


In [8]:
# Test decoder
decoder_outputs = decoder(encoder_output=encoder_output, max_length_trg=max_length_trg,max_length_src=max_length_src,mask=src_pad_mask,prev_hidden=None,target_tensor=target_tensor)
print("Decoder outputs shape:", decoder_outputs.shape)

True
no teacher forcing
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.Size([128, 10])
torch.Size([128, 1, 10])
torch.Size([128, 10, 512])
True
teacher forcing
input torch.Size([128, 1])
queries torch.Size([128, 512])
keys torch.Size([128, 512])
torch.S

In [ ]:

# Test attention
context = attention(encoder_output, h, src_pad_mask)
print("Attention context shape:", context.shape)

NameError: name 'attention' is not defined

In [ ]:
prev_hidden = torch.randint(0,10,(2,128,256))
first_hidden = torch.permute(prev_hidden,(1,0,2))[:,1,:].squeeze(1)
print(prev_hidden.shape)
print(first_hidden.shape)


torch.Size([2, 128, 256])
torch.Size([128, 256])


In [10]:
# Define the parameters for the RNN Encoder
source_vocab_size = 100  # example vocabulary size
rnn_hidden_size = 64  # example hidden size
embedding_size = 128  # example embedding size
num_rnn_layers = 3  # example number of LSTM layers
dropout_rate = 0.2  # example dropout rate
bidirectional = False  # example bidirectional flag
rnn_type = 'lstm'  # example RNN type

# Create an instance of the RNN_Encoder
encoder = RNN_Encoder(source_vocab_size, rnn_hidden_size, embedding_size,
                      num_rnn_layers, dropout_rate, bidirectional, rnn_type)

# Create a sample input sequence
batch_size = 3
max_sequence_length = 10
padded_src = torch.randint(0, source_vocab_size, (batch_size, max_sequence_length))
src_lens = torch.tensor([10, 8, 6])  # example sequence lengths

# Test the RNN Encoder
padded_output, state = encoder(padded_src, src_lens)

# Print the shapes of the output and state tensors
print('input shape:', padded_src.shape)
print("Output shape:", padded_output.shape)
print("State shape:", state[0].shape, state[1].shape)

input shape: torch.Size([3, 10])
Output shape: torch.Size([3, 10, 64])
State shape: torch.Size([3, 3, 64]) torch.Size([3, 3, 64])
tensor([-0.0115,  0.0563,  0.0280, -0.0098,  0.0003, -0.0372,  0.0252, -0.0451,
        -0.0238,  0.0323, -0.0252,  0.0258, -0.0462, -0.0223, -0.0621,  0.0325,
        -0.0194,  0.0262, -0.0080,  0.0110, -0.0335, -0.0308,  0.0039, -0.0148,
         0.0062, -0.0333,  0.0027, -0.0421,  0.0095, -0.0013, -0.0249,  0.0310,
         0.0249,  0.0152, -0.0404,  0.0111,  0.0374,  0.0660, -0.0249, -0.0481,
        -0.0019, -0.0387,  0.0057, -0.0087,  0.0349,  0.0389, -0.0058,  0.0180,
         0.0196, -0.0174, -0.0057,  0.0024, -0.0308, -0.0124, -0.0573,  0.0419,
        -0.0002,  0.0400,  0.0478,  0.0072, -0.0559, -0.0061, -0.0416,  0.0136],
       grad_fn=<SelectBackward0>)
